In [1]:
import pandas as pd

In [11]:
df = pd.read_csv("паспорт хорошего русского - песня скомпилированная рифма AABB.csv",)

In [14]:
df

,text
0,"[Война против Украины – грех страшной силы, Вы..."
1,[Путинский режим вызывает ненавись в наших гла...
2,"[Власть империи внутри и за пределами земель, ..."
3,[Политузников и пленных надо освободить скорей...
4,"[Солидарность голосов наших решительна, Нет и..."
5,"[Наша Декларация ценность воли разделяя,, Уваж..."
6,"[Вместе стоим в рядах антивоенных, Публичному ..."


In [13]:
def get_item(text):
    arr = text.split("\n")
    return list(map(lambda x: x if x[-1].isalpha() else x[:-1], arr))
    
#get_item(df.iloc[0].values[0])
df["text"] = df.apply(lambda raw:get_item(raw.iloc[0]), axis=1)

In [5]:
def get_rhyme_score(text1, text2):

    #This function counts only the direct equals of symbols from the end of phrases 
    #and isn`t aware of fonetical equals
    
    n = 1
    while (n < len(text1) or n < len(text2)):
        c1 = text1[-n]
        c2 = text2[-n] 
        #print(c1, c2)
        if c1 == c2:
            n = n+1
        else:
            break
    return n - 1

In [6]:
"A" , get_rhyme_score(df[0][0], df[0][1]), get_rhyme_score(df[0][0], df[0][2]), get_rhyme_score(df[0][0], df[0][3])

('A', 0, 3, 0)

In [7]:
"B" , get_rhyme_score(df[0][1], df[0][2]), get_rhyme_score(df[0][1], df[0][3])

('B', 0, 2)

In [8]:
def get_rhyme_score_of_quatrain(l4_quatrain):
    l1 = l4_quatrain
    letters = "ABCD"
    n = len(letters)
    scores = [None] * n 
    for i in range(0,n):
        current_letter = letters[i]
        if scores[i] is None: 
#            scores[i] = {current_letter:100}
            scores[i] = {current_letter}
        for j in range(i+1,n):
            score = get_rhyme_score(l1[i], l1[j])
            if (score > 0):
                scores[j] = {current_letter:score}
    return scores

In [9]:
l1 = df.iloc[6]
l1 , get_rhyme_score_of_quatrain(l1)

(['Вместе стоим в рядах антивоенных',
  'Публичному согласию мы знаем цену',
  'Солидарность и единство нам важны как никогда',
  'Из пепла восстанут наши города'],
 [{'A'}, {'B'}, {'C'}, {'C': 2}])

In [16]:
df["scores"] = df.apply(lambda raw: get_rhyme_score_of_quatrain(raw.iloc[0]), axis = 1)

In [17]:
df

,text,scores
0,"[Война против Украины – грех страшной силы, Вы...","[{A}, {B}, {'A': 3}, {'B': 2}]"
1,[Путинский режим вызывает ненавись в наших гла...,"[{A}, {B}, {'A': 1}, {'C': 2}]"
2,"[Власть империи внутри и за пределами земель, ...","[{A}, {'A': 1}, {C}, {'C': 2}]"
3,[Политузников и пленных надо освободить скорей...,"[{A}, {'A': 2}, {C}, {'C': 1}]"
4,"[Солидарность голосов наших решительна, Нет и...","[{A}, {'A': 2}, {C}, {'C': 2}]"
5,"[Наша Декларация ценность воли разделяя,, Уваж...","[{A}, {B}, {'A': 1}, {D}]"
6,"[Вместе стоим в рядах антивоенных, Публичному ...","[{A}, {B}, {C}, {'C': 2}]"


### Let`s control the different cases:

In [21]:
df.iloc[0].values
#here is Ok, ABAB

array([list(['Война против Украины – грех страшной силы', 'Выводить войска, искуплять вину  теперь такие наши дела', 'без вольной Украины мы сами на краю могилы', 'Мир и справедливость пока не сгорели мы дотла']),
       list([{'A'}, {'B'}, {'A': 3}, {'B': 2}])], dtype=object)

In [24]:
df.iloc[1].values
#here is ABAC, should be AABB
#because phonetically ах ~ аш

array([list(['Путинский режим вызывает ненавись в наших глазах', 'Ликвидация режима это долго, но верный выбор наш', 'Мы мечтаем о России, где права есть у всех', 'Народ без Путина, в этом первый наш успех']),
       list([{'A'}, {'B'}, {'A': 1}, {'C': 2}])], dtype=object)

In [25]:
df.iloc[6].values
#here is ABCC, should be AABB 
#because phonetically ных ~ ну

array([list(['Вместе стоим в рядах антивоенных', 'Публичному согласию мы знаем цену', 'Солидарность и единство нам важны как никогда', 'Из пепла восстанут наши города']),
       list([{'A'}, {'B'}, {'C'}, {'C': 2}])], dtype=object)

### Conclusion: the get_rhyme_score(text1, text2) should be extended for fonetical equal phrases